In [2]:
# Step 1: Import Required Libraries
import pandas as pd

# Step 2: Load Excel File
df = pd.read_excel('Railway_info.csv.xlsx')  # Make sure this file is in the same folder

# Step 3: View Column Names
print("Available columns:", df.columns.tolist())


Available columns: ['Train_No', 'Train_Name', 'Source_Station_Name', 'Destination_Station_Name', 'days']


In [3]:
# Clean column names to avoid KeyError
df.columns = df.columns.str.strip()

# Print to confirm
print(df.columns.tolist())

['Train_No', 'Train_Name', 'Source_Station_Name', 'Destination_Station_Name', 'days']


In [4]:
# Filter trains that operate only on Saturdays
Saturday_trains = df[df['days'] == 'Saturday']
print("Trains running on Saturday:\n", Saturday_trains[['Train_No', 'Train_Name', 'Source_Station_Name', 'Destination_Station_Name']].head())

# Extract trains starting from a specific station, e.g., 'DELHI'
from_delhi = df[df['Source_Station_Name'].str.upper() == 'DELHI']
print("\nTrains starting from DELHI:\n", from_delhi[['Train_No', 'Train_Name', 'Source_Station_Name', 'Destination_Station_Name']].head())

Trains running on Saturday:
     Train_No    Train_Name Source_Station_Name Destination_Station_Name
0        107  SWV-MAO-VLNK     SAWANTWADI ROAD              MADGOAN JN.
4        401  BSB BHARATDA          AURANGABAD             VARANASI JN.
21      1196  NGP-KRMI SPL      NAGPUR JN.(CR)                  KARMALI
28      1706   JBP-BDTS SF            JABALPUR          BANDRA TERMINUS
45      2834  SRC-RJT SF A     SANTRAGACHI JN.                   RAJKOT

Trains starting from DELHI:
 Empty DataFrame
Columns: [Train_No, Train_Name, Source_Station_Name, Destination_Station_Name]
Index: []


In [5]:
source_station_counts = df.groupby('Source_Station_Name').size().reset_index(name='Train_Count')
print(source_station_counts.sort_values(by='Train_Count', ascending=False).head())

    Source_Station_Name  Train_Count
224          CST-MUMBAI          513
773             SEALDAH          372
201       CHENNAI BEACH          339
373          HOWRAH JN.          338
426           KALYAN JN          285


In [6]:
# Calculate average number of operating days for trains from each source station
df['Days_Operating'] = df['days'].apply(lambda x: len(str(x).split()))
avg_trains_per_day = df.groupby('Source_Station_Name')['Days_Operating'].mean().reset_index(name='Avg_Trains_Per_Day')
print(avg_trains_per_day.sort_values(by='Avg_Trains_Per_Day', ascending=False).head())

    Source_Station_Name  Avg_Trains_Per_Day
920      YESVANTPUR JN.                 1.0
0          ABHANPUR JN.                 1.0
1                ABOHAR                 1.0
2              ABU ROAD                 1.0
3          ACHHNERA JN.                 1.0


In [7]:
# Define weekday and weekend sets
weekdays = {'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'}
weekends = {'Saturday', 'Sunday'}

# Function to categorize day types
def categorize_days(day_string):
    if pd.isna(day_string):
        return 'Unknown'
    day_set = set(day_string.strip().split())
    if day_set & weekends and not day_set & weekdays:
        return 'Weekend'
    elif day_set & weekdays and not day_set & weekends:
        return 'Weekday'
    elif day_set & weekdays and day_set & weekends:
        return 'Both'
    else:
        return 'Unknown'

# Apply transformation
df['Day_Category'] = df['days'].apply(categorize_days)

# Show category counts
print(df['Day_Category'].value_counts())

# Show sample results
df[['Train_No', 'Train_Name', 'days', 'Day_Category']].head()

Day_Category
Weekday    7087
Weekend    2873
Unknown    1153
Name: count, dtype: int64


,Train_No,Train_Name,days,Day_Category
0,107,SWV-MAO-VLNK,Saturday,Weekend
1,108,VLNK-MAO-SWV,Friday,Weekday
2,128,MAO-KOP SPEC,Friday,Weekday
3,290,PALACE ON WH,Wednesday,Weekday
4,401,BSB BHARATDA,Saturday,Weekend
